# Labeling posts with Llama3-8b via Groq

We'll use the LiteLLM [Groq](https://litellm.vercel.app/docs/providers/groq) connection to connect to Groq and use that to label via Llama3-8b.


In [2]:
import json
import os

import pandas as pd

from ml_tooling.llm.inference import run_query

In [3]:
current_wd = os.getcwd()

### Let's load our data

In [4]:
with open("links_to_prompts_map_v2.json") as f:
    links_to_prompt_map = json.load(f)

In [5]:
links_prompts_lst: list[tuple[str, dict]] = [
    (link, prompt_dict)
    for (link, prompt_dict) in links_to_prompt_map.items()
]

Let's see what this looks like:

In [6]:
links_prompts_lst[0]

('https://bsky.app/profile/jbouie.bsky.social/post/3knqbtrdzrz2n',
 {'context_prompt': '\n\nPretend that you are a classifier that predicts whether a post has civic content or not. Civic refers to whether a given post is related to politics (government, elections, politicians, activism, etc.) or social issues (major issues that affect a large group of people, such as the economy, inequality, racism, education, immigration, human rights, the environment, etc.). We refer to any content that is classified as being either of these two categories as “civic”; otherwise they are not civic. Please classify the following text denoted in <text> as "civic" or "not civic". \n\nThen, if the post is civic, classify the text based on the political lean of the opinion or argument it presents. Your options are \'left-leaning\', \'moderate\', \'right-leaning\', or \'unclear\'. You are analyzing text that has been pre-identified as \'political\' in nature. If the text is not civic, return "unclear".\n\nT

#### Llama3-8b

Let's store all the. Llama3-8b results in one dictionary. Let's initialize that dictionary with the links and prompts from the input, and then for the cases where the prompt with context equals the prompt without context, we only run the query once in order to avoid duplication.

In [8]:
llama3_8b_results: dict = {}

In [9]:
for (link, prompt_dict) in links_to_prompt_map.items():
    llama3_8b_results[link] = prompt_dict
    llama3_8b_results[link]["context_llm_result"] = ""
    llama3_8b_results[link]["no_context_llm_result"] = ""

Let's create a mapping of links to prompts for both context and without context prompts, so we can just iterate through those during inference.

In [19]:
# we only want to run the context classification for posts
# that have context in the first place. If the prompts between
# the context and no-context cases are the 
link_to_context_prompt_map: dict = {
    link: prompt_dict["context_prompt"]
    for (link, prompt_dict) in llama3_8b_results.items()
    if not prompt_dict["prompts_are_equal"]
}

In [20]:
# spot-check and see if the prompt indeed has context.
# It appears to be correct. These should all have prompts
print(list(link_to_context_prompt_map.items())[0][0])
print(list(link_to_context_prompt_map.items())[0][1])

https://bsky.app/profile/jbouie.bsky.social/post/3knqbtrdzrz2n


Pretend that you are a classifier that predicts whether a post has civic content or not. Civic refers to whether a given post is related to politics (government, elections, politicians, activism, etc.) or social issues (major issues that affect a large group of people, such as the economy, inequality, racism, education, immigration, human rights, the environment, etc.). We refer to any content that is classified as being either of these two categories as “civic”; otherwise they are not civic. Please classify the following text denoted in <text> as "civic" or "not civic". 

Then, if the post is civic, classify the text based on the political lean of the opinion or argument it presents. Your options are 'left-leaning', 'moderate', 'right-leaning', or 'unclear'. You are analyzing text that has been pre-identified as 'political' in nature. If the text is not civic, return "unclear".

Think through your response step by step.


In [22]:
# we have 204 posts with context
len(link_to_context_prompt_map)

204

Now let's get the prompts without context

In [21]:
# we run the no-context prompts for all links
link_to_no_context_prompt_map: dict = {
    link: prompt_dict["no_context_prompt"]
    for (link, prompt_dict) in llama3_8b_results.items()
}

In [23]:
# spot-check and see if the prompt indeed has context.
# It appears to be correct.
print(list(link_to_context_prompt_map.items())[0][0])
print(list(link_to_context_prompt_map.items())[0][1])

https://bsky.app/profile/jbouie.bsky.social/post/3knqbtrdzrz2n


Pretend that you are a classifier that predicts whether a post has civic content or not. Civic refers to whether a given post is related to politics (government, elections, politicians, activism, etc.) or social issues (major issues that affect a large group of people, such as the economy, inequality, racism, education, immigration, human rights, the environment, etc.). We refer to any content that is classified as being either of these two categories as “civic”; otherwise they are not civic. Please classify the following text denoted in <text> as "civic" or "not civic". 

Then, if the post is civic, classify the text based on the political lean of the opinion or argument it presents. Your options are 'left-leaning', 'moderate', 'right-leaning', or 'unclear'. You are analyzing text that has been pre-identified as 'political' in nature. If the text is not civic, return "unclear".

Think through your response step by step.


In [24]:
# 354 without context.
len(link_to_no_context_prompt_map)

354

##### With context

Let's run inference for llama3-8b with context and export the results. Let's then compare it to the ground-truth labels.

In [ ]:
llama3_8b_with_context_llm_results: list[str] = []

In [ ]:
# run queries against Llama3-8b model. This will take a while since we're
# making a lot of requests to Groq. Takes ~30 minutes.
for idx, prompt in enumerate(prompts_lst):
    if idx % 50 == 0:
        print(f"Processing post {idx + 1} of {total_prompts}")
    try:
        result = run_query(prompt=prompt, model_name="Llama3-8b (via Groq)")
        llama3_8b_with_context_llm_results.append(result)
    except Exception as e:
        # for the ones that failed, this happens because we set a requirement
        # that it must be valid JSON. Some of the responses are not JSON, so
        # Groq throws an error and tells us that the result wasn't JSON.
        print(f"Error with post {post['link']}: {e}")
        llama3_8b_with_context_llm_results.append("")
        continue